In [3]:
from Bio import Entrez
import pandas as pd
from xml.dom.minidom import parse
import xml.dom.minidom
import pickle
import json
from pymed import PubMed
import requests
import datetime
import os
import pyspark
from pyspark import SparkConf, SparkContext
import sys
from collections import defaultdict

pubmed = PubMed(tool="MyTool", email="jefflovejoy1314@gmail.com")

In [9]:
query_string = '("neoplasms"[MeSH Terms] OR "neoplasms"[All Fields] OR "cancer"[All Fields]) AND ("2009/11/24"[PDat] : "2019/11/21"[PDat])'
results = pubmed.query(query_string, max_results=3965593)

In [ ]:
file_number = 0

info_dict = {'pubmed_id': [], 'authors': [], 'keywords':[], 'abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 
             'journal':[]}


for i in results:
#     print(len(info_dict['pubmed_id']))
    for key in list(info_dict.keys()):
        try:
            if key == 'authors':
                au = i.authors
                auinfo = []
                for x in au: 
                    name = str(x['firstname']) + ' ' + str(x['lastname'])
                    affiliation = x['affiliation']
                    auinfo.append(f'{name}-{affiliation}')
                info_dict[key].append(';'.join(auinfo))
            elif key == 'pubmed_id':
                info_dict[key].append(str(i.pubmed_id.split('\n')[0]))
            else:
                info_dict[key].append(getattr(i,key))

        except:
            info_dict[key].append(None)
            
    if len(info_dict['pubmed_id']) == 15000:
        pd.DataFrame(info_dict).to_csv(f'cancer/cancer_{file_number}.csv')
        info_dict = {'pubmed_id': [], 'authors': [], 'keywords':[], 'abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 
             'journal':[]}
        file_number += 1

        print(file_number)
        

In [ ]:
sc = pyspark.SparkContext.getOrCreate()
sc.textFile('gs://project_rong/cancer/gcp_codes.txt')

pd.read_csv('cancer/cancer_100.csv')
! gsutil cp cancer/cancer_1.csv gs://project_rong/cancer/cancer_1.csv

In [ ]:
# combine all the files
total_file = pd.read_csv('cancer/cancer_0.csv')

for r,d,f in os.walk('cancer'): 
    for fn in f:
        if fn == 'cancer_0.csv':
            continue
        else: 
            file1 = pd.read_csv(f'cancer/{fn}')
            total_file = pd.concat([total_file, file1], ignore_index= True)
    
        
pd.DataFrame(total_file).to_csv('cancer/cancer_total.csv')

In [ ]:
! gsutil cp cancer/cancer_total.csv gs://project_rong/cancer/cancer_total.csv
! gsutil cp gs://project_rong/bigdata-d7b584548bdd.json bigdata-d7b584548bdd.json

In [ ]:
# parse info for authors
import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('bigdata-d7b584548bdd.json')
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "bigdata-259800"


au_pair = []

for au_group in useful_au:
    au_pairlist = list(permutations([x.replace('\n', ' ') for x in au_group.split(";")],2))
    au_pair.extend(au_pairlist)

edges = pd.DataFrame({'source': [i[0] for i in au_pair], 'target':[i[1] for i in au_pair]})
nodes = pd.DataFrame({'au_nodes': au_nodes_list})


nodes.to_csv('cancer/au_nodes.csv')
pandas_gbq.to_gbq(edges, 'authorinfo.author_edges', project_id= "bigdata-259800")
pandas_gbq.to_gbq(nodes, 'authorinfo.author_nodes', project_id= "bigdata-259800")

